In [1]:
import numpy as np
import gemmi
import mdtraj as md
from matplotlib import pyplot as plt

In [2]:
# the "FAD frame" is:
#  +x from C10 to C9A
#  +y from N5 to N10
#  +z completing the right hand rule

# from Pauszek et al JACS 2016, 138, 14880−14889 DOI:10.1021/jacs.6b06449 

TDM_IN_FAD_FRAME_S01 = np.array([-0.3059, 0.3008, -0.0162])
TDM_IN_FAD_FRAME_S02 = np.array([-1.1889, 0.0081, -0.1393])

CHAIN_A_FAD_N5 = np.array([14.968,  13.196,  -4.538])
CHAIN_B_FAD_N5 = np.array([ 8.418, -13.783, -26.245])

In [3]:
def crystal_to_fad(trj, chain=0):

    if chain not in [0, 2]:
        raise ValueError('You want chain = 0 (A) or 2 (B)')

    approx_basis_vectors = []
    xyzs = []

    for i, atom in enumerate(["N10", "N5", "C9A", "C10"]):

        atom_sele = f'name {atom}'
        selection = f'(chainid {chain}) and (resname FDA) and ({atom_sele})'
        sele = trj.top.select(selection)

        assert len(sele) == 1, len(sele)

        xyzs.append(trj.xyz[0, sele[0], :])

    x_approx = xyzs[2] - xyzs[3]
    x_approx = x_approx / np.linalg.norm(x_approx)

    y_approx = xyzs[1] - xyzs[0]
    y_approx = y_approx / np.linalg.norm(y_approx)

    # this is approximate, but there is no way to do better from the information I have
    #   1. we assign (C10 -> C9A) as +x
    #   2. the, we assign (N10 -> N5) x (C10 -> C9A) as +z
    #   3. finally, (x) x (z) as +y, which should be approximately (C10 -> C9A)

    x = x_approx
    z = np.cross(x, y_approx)
    y = np.cross(z, x)

    assert np.abs(np.dot(y_approx, y)) > 0.99

    R = np.column_stack([x, y, z])
    det = np.linalg.det(R)
    assert np.abs(det - 1.0) < 0.001, det

    return R


def rotation_matrix_from_A(A):
    """
    For an orthorhombic system, we can write the A matrix as:

        A = R S I

    Where:
        -- I is the identity
        -- S is a (diagonal) scaling matrix that sets the length of each a/b/c/ axis
        -- R is a rotation matrix

    So, we'll simply write:

        R = A S^{-1}
    """

    if not A.shape == (3, 3):
        raise ValueError('A not 3x3', A.shape)

    # cell lengths in nm (CrystFEL uses nm)
    S = np.diag([70.200,  117.760,  170.470]) * 0.1
    R = np.dot(A, np.linalg.inv(S))

    # rotation matrices should have det(R) = 1
    det = np.linalg.det(R)
    assert np.abs(det - 1.0) < 0.05, det

    return R


def expand_p212121(vector):

    sg = gemmi.SpaceGroup('P212121')
    ops = sg.operations()

    set_of_vectors = []
    for op in ops:
        vt = np.dot( np.array(op.rot) / 24.0, vector)
        set_of_vectors.append(vt)

    return np.array(set_of_vectors)


def compute_tdm_Pz(tdm_vector_in_crystal_setting, A):
    """
    From a tdm vector, in the crystal setting, return the component perpindicular to z
      1. symmetry expand
      2. rotate
      3. project on z
      4. return Pz
    """

    tdm_hat = tdm_vector_in_crystal_setting / \
        np.linalg.norm(tdm_vector_in_crystal_setting)
    
    z_hat = np.array([0.0, 0.0, 1.0])

    tdm_set = expand_p212121(tdm_hat)
    assert tdm_set.shape[-1] == 3
    assert np.all( np.abs(np.linalg.norm(tdm_set, axis=-1) - 1) < 0.0001)

    R = rotation_matrix_from_A(A)

    rotated_tdms = np.dot(tdm_set, R.T)
    projectd_tdms = np.dot(rotated_tdms, z_hat)

    Pz = np.sum(1.0 - np.square(projectd_tdms)) / float(tdm_set.shape[0])

    return Pz


### TDM

Logic. Given TDM $\mu$ in the FAD frame of reference, we need to compute the set of TDMs in the experimental frame. To do this, we:

1. for each chain: compute the rotation matrices to go from the crystal frame to the frame of FAD (2 rotation matrices)
2. compute the TDM for each chain in the crystal frame (2 vectors)
3. symmetry-expand these vectors (4 + 4 vectors)
4. rotate these vectors using the A-matrix into the experimental frame

Then we can simply compute the component of the TDM that is in the (circular) polarization plane as:

$$
\sum_i 1 - (\vec{\mu}_i \cdot \vec{z})^2
$$

In code, we refer to this quantity as "Pz"

NB! The A-matices were computed such that (a,b,c) formed the ROWS of the matrix, but here we transpose them such that they form the COLUMNS, which is more standard.

First, pymol validation of placement of TDMs in the crystal frame...

In [4]:
trj = md.load_pdb('../pltest.pdb')
R = crystal_to_fad(trj, chain=2)
print(R)

tdm = np.dot(R, TDM_IN_FAD_FRAME_S02)

[[ 0.6257882  -0.35356882  0.6952449 ]
 [ 0.6022435   0.7854614  -0.1426178 ]
 [-0.49567318  0.50795525  0.704467  ]]


In [5]:
atom = CHAIN_B_FAD_N5

print(f"pseudoatom pt1, pos={[a for a in atom]}")
print(f"pseudoatom pt2, pos={[a for a in 10*tdm+atom]}")
print("distance /pt1, /pt2")
print("set dash_gap, 0")


pseudoatom pt1, pos=[8.418, -13.783, -26.245]
pseudoatom pt2, pos=[-0.01911128225922809, -20.680783789828418, -21.292119720935823]
distance /pt1, /pt2
set dash_gap, 0


Small tests for the projection code

In [6]:
x = np.array([1.0, 0.0, 0.0])
z = np.array([0.0, 0.0, 1.0])

non_rotating_A = np.diag([70.200,  117.760,  170.470]) * 0.1

assert np.abs(compute_tdm_Pz(x, non_rotating_A) - 1.0) < 0.001
assert np.abs(compute_tdm_Pz(z, non_rotating_A) - 0.0) < 0.001
assert np.abs(compute_tdm_Pz(x+z, non_rotating_A) - 0.5) < 0.001

### do the calculation

In [7]:
a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_TD_all.npy'
#a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_100us_dark.npy'
#a_matrix_file = '/Users/tjlane/Desktop/PL-workshop/signals_chain_A_vs_B/orient/crystal_Amatrices_100us_l+d.npy'

# NB transpose so that columns are a/b/c vectors
A_matrices = np.transpose(np.load(a_matrix_file), axes=(0,2,1))
print(A_matrices.shape)

(35113, 3, 3)


In [9]:
tdm_names = ['S01', 'S02']

print('Projection Magnitudes')
print('')
print('TDM\tChain A\tChain B')
print('---------------------')

for i,tdm_fad_frame in enumerate([TDM_IN_FAD_FRAME_S01, TDM_IN_FAD_FRAME_S02]):

    tdm_chainA = np.dot(crystal_to_fad(trj, chain=0), tdm_fad_frame)
    tdm_chainB = np.dot(crystal_to_fad(trj, chain=2), tdm_fad_frame)

    chainA_projection = np.mean([compute_tdm_Pz(tdm_chainA, A) for A in A_matrices])
    chainB_projection = np.mean([compute_tdm_Pz(tdm_chainB, A) for A in A_matrices])

    print("%s\t%.3f\t%.3f" % (tdm_names[i], chainA_projection, chainB_projection))

Projection Magnitudes

TDM	Chain A	Chain B
---------------------
S01	0.599	0.600
S02	0.744	0.699


### Reference calculation -- randomly oriented TDM (e.g. in solution)

Recall $z = \cos \theta$ in spherical coordinates and that the surface area of the unit sphere is $4 \pi$. We want to integrate the function $ 1 - (\hat{\mu} \cdot \hat{z})^2 = 1 - \cos^2 \theta $ over the surface of the sphere. 

$$
\frac{1}{4 \pi} \int_0^{2 \pi} \int_0^{\pi} (1 - \cos^2 \theta) \sin \theta d\theta d\phi = 2/3
$$